# 智能水务平台综合案例教程

欢迎来到智能水务平台 (SWP) 的综合案例教程！

在本教程中，我们将端到端地走过一个典型的水务管理场景，将数据输入、模型仿真、性能评估和智能预测等多个环节串联起来。我们将使用本平台最新开发的功能，包括：

1.  **数据清洗与准备** (`swp.data_processing.cleaner`)
2.  **降雨-径流模型** (`swp.simulation_identification.physical_objects.RainfallRunoff`)
3.  **模型性能评估** (`swp.data_processing.evaluator`)
4.  **ARIMA 预测模型** (`swp.local_agents.prediction.ARIMAForecaster`)

---

## 案例背景

我们负责管理一个中型水库。该水库的主要水源来自一个山区汇水区，其径流由降雨驱动。水库下游有一个城市需要我们稳定供水。

**我们的任务是：**

1.  建立一个能够模拟“降雨 -> 汇水区产流 -> 水库入流”过程的数字孪生模型。
2.  使用历史观测数据，检验我们模型的准确性。
3.  利用历史入库流量数据，训练一个预测模型，以预报未来的入库流量。
4.  探讨如何利用预测信息，来更智能地制定水库的放水策略。

### 准备工作

首先，我们导入所有需要的库和模块。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 导入我们新开发的模块
from swp.data_processing.cleaner import fill_missing_with_interpolation
from swp.data_processing.evaluator import calculate_nse, calculate_kge, calculate_rmse

# 导入平台的核心组件
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus

# 导入物理对象和智能体
from swp.simulation_identification.physical_objects.rainfall_runoff import RainfallRunoff
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.disturbances.rainfall_agent import RainfallAgent
from swp.local_agents.prediction.arima_forecaster import ARIMAForecaster

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')

## 第一部分：数据准备与清洗

我们有两份历史数据：一份是汇水区的降雨强度数据 (`rainfall_data.csv`)，另一份是水库的实际观测入库流量数据 (`historical_inflow.csv`)。后者将作为我们评估模型性能的“黄金标准”。

In [ ]:
rainfall_df = pd.read_csv('../../data/rainfall_data.csv')
observed_inflow_df = pd.read_csv('../../data/historical_inflow.csv')

# 为了演示数据清洗功能，我们手动引入一些缺失值 (NaN)
observed_inflow_df_missing = observed_inflow_df.copy()
observed_inflow_df_missing.loc[[3, 7], 'inflow'] = np.nan

print("带有缺失值的观测数据:")
print(observed_inflow_df_missing)

In [ ]:
# 使用我们新的清洗工具来填充缺失值
cleaned_inflow = fill_missing_with_interpolation(observed_inflow_df_missing['inflow'])

print("\n清洗后的数据:")
print(cleaned_inflow)

## 第二部分：系统建模与仿真

现在，我们开始搭建数字孪生系统。这个系统包含三个核心物理组件：
1.  `RainfallRunoff`: 模拟汇水区，接收降雨数据并计算产生的径流。
2.  `Reservoir`: 我们的水库，接收来自汇水区的径流。
3.  `Gate`: 水库的出口闸门，我们设定一个固定的开启程度来放水。

此外，我们还需要一个`RainfallAgent`来读取`rainfall_data.csv`并通过消息总线`MessageBus`将数据“播放”给`RainfallRunoff`模型。

In [ ]:
# 1. 初始化仿真核心组件
harness = SimulationHarness(config={'dt': 10, 'duration': 100}) # 仿真100秒，每10秒一步
bus = harness.message_bus # 使用Harness内置的总线

# 2. 定义组件参数
rainfall_topic = "weather/catchment/rainfall"
runoff_params = {'catchment_area': 2.5e7, 'runoff_coefficient': 0.5}
reservoir_params = {'surface_area': 1.5e6}
gate_params = {'width': 5, 'discharge_coeff': 0.8}

# 3. 创建物理组件实例
runoff_model = RainfallRunoff("catchment_a", runoff_params, bus, rainfall_topic)
reservoir_model = Reservoir("main_reservoir", {'volume': 25e6}, reservoir_params)
gate_model = Gate("outlet_gate", {'opening': 0.6}, gate_params)

# 4. 创建数据播放智能体
# 这里我们简化一下，直接在循环里发消息，而不是用一个完整的RainfallAgent

# 5. 将组件添加到仿真平台并定义连接关系
harness.add_component(runoff_model)
harness.add_component(reservoir_model)
harness.add_component(gate_model)

harness.add_connection("catchment_a", "main_reservoir")
harness.add_connection("main_reservoir", "outlet_gate")

# 6. 构建仿真
harness.build()

print("仿真平台构建完成！")

In [ ]:
# 7. 运行仿真
print("开始运行仿真...")

simulated_inflow_history = []

for t in range(0, 101, 10):
    # 在每个时间步，发布对应的降雨数据
    current_rain = rainfall_df[rainfall_df['time'] == t]['rainfall_intensity'].values[0]
    bus.publish(rainfall_topic, {'rainfall_intensity': current_rain})
    
    # 运行一步仿真
    harness.step()
    
    # 记录模拟出的径流（即水库的入流）
    simulated_inflow = runoff_model.get_state()['outflow']
    simulated_inflow_history.append(simulated_inflow)

print("\n仿真结束！")

# 整理并打印结果
results_df = pd.DataFrame({
    'Time': rainfall_df['time'],
    'Observed_Inflow': observed_inflow_df['inflow'],
    'Simulated_Inflow': simulated_inflow_history
})

print(results_df)

## 第三部分：评价与诊断

我们的`RainfallRunoff`模型表现如何？现在我们使用`evaluator`模块中的工具来量化它的性能。

In [ ]:
sim = results_df['Simulated_Inflow']
obs = results_df['Observed_Inflow']

rmse = calculate_rmse(sim, obs)
nse = calculate_nse(sim, obs)
kge = calculate_kge(sim, obs)

print(f"模型性能评估:")
print(f"  均方根误差 (RMSE): {rmse:.2f}")
print(f"  纳什效率系数 (NSE): {nse:.2f}")
print(f"  克林-古普塔效率 (KGE): {kge:.2f}")

# 可视化对比
plt.figure(figsize=(12, 6))
plt.plot(results_df['Time'], obs, 'o-', label='观测入流 (Observed)')
plt.plot(results_df['Time'], sim, 's--', label='模拟入流 (Simulated)')
plt.title('模拟入流 vs 观测入流')
plt.xlabel('时间 (s)')
plt.ylabel('流量 (m^3/s)')
plt.legend()
plt.show()

## 第四部分：预测与预警

现在，假设我们的`RainfallRunoff`模型足够准确，我们可以信任它的输出。接下来，我们将使用`ARIMAForecaster`来预测未来的入库流量。

我们将使用清洗后的历史观测数据 (`cleaned_inflow`) 来训练预测模型。

In [ ]:
# 1. 配置并创建预测智能体
arima_config = {
    "observation_topic": "data/inflow_history",
    "observation_key": "value",
    "forecast_topic": "forecast/inflow",
    "history_size": 50, # 使用过去50个数据点
    "arima_order": (2, 1, 1), # 一个常用的ARIMA参数配置
    "forecast_steps": 5, # 预测未来5个步长
    "refit_interval": 10 # 每10个新数据点重新训练一次模型
}

forecaster_bus = MessageBus()
forecaster = ARIMAForecaster("inflow_forecaster", forecaster_bus, arima_config)

# 2. 准备一个Listener来接收预测结果
forecast_results = []
forecaster_bus.subscribe(arima_config["forecast_topic"], lambda msg: forecast_results.append(msg))

# 3. 喂给模型历史数据并触发预测
# 我们将历史数据的前半部分作为训练数据
training_data = cleaned_inflow.iloc[:11]
for val in training_data:
    forecaster_bus.publish(arima_config["observation_topic"], {"value": val})

# 运行agent来训练并生成预测
forecaster.run(current_time=100)

print("预测完成!")
if forecast_results:
    print("预测结果:", forecast_results[0]['values'])


## 第五部分：调度与控制

在上一部分，我们成功预测了未来5个时间步的入库流量。这些信息对于水库的调度至关重要。

一个传统的控制策略可能是：当水库水位超过某个阈值时，就开大闸门；低于某个阈值时，就关小闸门。这是一种**被动式**的反馈控制。

而有了预测信息，我们就可以实现**主动式**的预判控制。例如：

- **预见性防洪**：如果`ARIMAForecaster`预测到未来几小时将有一场大流量入库，即使当前水位尚在安全范围，调度中心也可以决定提前、逐步地开大闸门，预先腾出库容来迎接洪水，避免到时候手忙脚乱，被迫瞬间全力泄洪。
- **优化发电/供水**：如果预测未来一段时间是枯水期，入流持续偏小，调度中心可以提前减小发电或供水流量，以保证长期、稳定的水源供应。

这些复杂的决策逻辑，正是`swp.central_coordination.dispatch.CentralDispatcher` 和更高级的控制智能体（如MPC控制器）未来需要实现的功能。我们的预测模块为这些高级功能提供了关键的数据支持。

## 总结

恭喜你完成了本次综合教程！

通过这个案例，我们：
1.  学习了如何使用`swp.data_processing`模块来清洗数据和评估模型。
2.  成功搭建并运行了一个包含自定义`RainfallRunoff`模型的数字孪生系统。
3.  掌握了如何使用`ARIMAForecaster`来对关键变量进行定量预测。
4.  理解了预测信息对于实现高级智能调度的重要意义。

这展示了智能水务平台作为一个可扩展的“母机”，如何将不同的功能模块组合起来，解决复杂的水务管理问题。